In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras as keras
from keras.preprocessing.image import ImageDataGenerator
from scipy.io import loadmat
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import numpy as np
import glob
import os
from PIL import Image
from sklearn import preprocessing
from sklearn.utils import shuffle
import cv2

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
data_dir = "animedata/full_dataset/animefaces"
list_ds = tf.data.Dataset.list_files(str(data_dir+'*/*'))
IMG_HEIGHT = 72
IMG_WIDTH = 72
BATCH_SIZE = 32
#CLASS_NAMES = ['animefaces']
AUTOTUNE=tf.data.experimental.AUTOTUNE

In [ ]:
for f in list_ds.take(5):
    print(f)

In [ ]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
@tf.autograph.experimental.do_not_convert
def process_path(file_path):
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img
@tf.autograph.experimental.do_not_convert
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    # use caching if the data is too large for the memory
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()
    # shuffle
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()
    ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

In [ ]:
full_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
# check if the image shape is correct
for image in full_ds.take(1):
    print("Input Image shape: ", image.numpy().shape)

In [ ]:
steps_per_epoch = None
def show_batch(image_batch):
    plt.figure(figsize=(25,25))
    for n in range(BATCH_SIZE):
        ax = plt.subplot(8,BATCH_SIZE/8,n+1)
        plt.imshow(image_batch[n])
        plt.axis('off')

In [ ]:
full_ds = prepare_for_training(full_ds)
#image_batch = next(iter(full_ds))
#show_batch(image_batch.numpy())

In [ ]:
#full_ds = full_ds.shuffle()
# split train and test if required
train_ds = full_ds.take(50000)
test_ds = full_ds.skip(50000)
train_np = tfds.as_numpy(train_ds)
it = iter(train_ds)
train_ds

In [ ]:
def def_discriminator(shape=(72,72,3)):
    inputs = keras.Input(shape)
    x = keras.layers.Conv2D(filters = 100, kernel_size = (4,4),strides = 2, padding = 'same', activation = "relu")(inputs)
    #x = keras.layers.MaxPooling2D((2,2), padding = 'same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(filters = 100, kernel_size = (4,4),strides = 2, padding = 'same', activation = "relu")(x)
    #x = keras.layers.MaxPooling2D((2,2), padding = 'same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(filters = 100, kernel_size = (2,2),strides = 2, padding = 'same', activation = "relu")(x)
    x = keras.layers.MaxPooling2D((2,2), padding = 'same')(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(50)(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.BatchNormalization()(x)
    output = keras.layers.Dense(2, activation = 'softmax')(x)
    model = keras.Model(inputs,output)
    opt = keras.optimizers.Adam(learning_rate = 0.0002, beta_1 = 0.5)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
disc_model = def_discriminator()
disc_model.summary()

In [ ]:
def def_generator(latent_dim):
    inputs = keras.Input((latent_dim,))
    x = keras.layers.Dense(9*9*100)(inputs)
    x = keras.layers.LeakyReLU(alpha=0.2)(x)
    x = keras.layers.Reshape((9,9,100))(x)
    x = keras.layers.Conv2DTranspose(140,(2,2),strides=(2,2),padding='same')(x)
    x = keras.layers.LeakyReLU(alpha=0.2)(x)
    #x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Conv2DTranspose(140,(4,4),strides=(2,2),padding='same')(x)
    x = keras.layers.LeakyReLU(alpha=0.2)(x)
    x = keras.layers.Conv2DTranspose(100,(5,5),strides=(2,2),padding='same')(x)
    x = keras.layers.LeakyReLU(alpha=0.2)(x)
    #x = keras.layers.Dropout(0.4)(x)
    output = keras.layers.Conv2D(3,(4,4),activation='tanh',padding='same')(x)
    model = keras.Model(inputs,output)
    return model
latent_dim = 128
gen_model = def_generator(latent_dim)
gen_model.summary()

In [ ]:
@tf.autograph.experimental.do_not_convert
def def_gan(gen_model,disc_model):
    disc_model.trainable = False
    model = keras.Sequential()
    model.add(gen_model)
    model.add(disc_model)
    opt = keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='categorical_crossentropy', optimizer=opt)
    return model
gan_model = def_gan(gen_model, disc_model)

In [ ]:
def select_real_samples(dataset, n_samples):
    # choose random instances
    X = np.array(dataset[:n_samples])
    # generate 'real' class labels (1)
    y = np.full((n_samples, 2),[0,1])
    return X, y
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = np.random.randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
def generate_fake_samples(gen_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = gen_model.predict(x_input)
    # create 'fake' class labels (0)
    y = np.full((n_samples, 2),[0,0])
    return X, y

In [ ]:
keras.models.Model.tr = tf.autograph.experimental.do_not_convert(func=keras.models.Model.train_on_batch)

In [ ]:
n_batches = 50000/BATCH_SIZE
epochs = 10
bat_per_epo = int(n_batches/epochs)
# half batch - to train disc one half with fake and other half with real
half_batch = int(BATCH_SIZE/2)
# iterate over epochs
for i in range(epochs):
    # enumerate batches over the training set
    for j in range(bat_per_epo):
        # get randomly selected 'real' samples
        X_real, y_real = select_real_samples(next(iter(train_ds)), half_batch)
        # update discriminator model weights using real samples
        #d_loss1, acc1 = disc_model.tr(X_real, y_real)
        # generate 'fake' examples
        X_fake, y_fake = generate_fake_samples(gen_model, latent_dim, half_batch)
        # update discriminator model weights using fake samples
        #d_loss2, acc2 = disc_model.tr(X_fake, y_fake)
        X = np.r_[X_real,X_fake]
        y = np.r_[y_real,y_fake]
        X = tf.convert_to_tensor(X)
        y = tf.convert_to_tensor(y)
        #a,b = shuffle(X,y, random_state=0)
        d_loss1, acc1 = disc_model.train_on_batch(X, y)
        
        # we update generator 5 times per one training of disc

        for k in range(3):
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, BATCH_SIZE)
            # create inverted labels for the fake samples
            y_gan = np.full((BATCH_SIZE, 2),[0,1])
            # update the generator via the discriminator's error until disc loss increases
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
        
    # summarize loss after this epoch
    #print('>%d, %d/%d, d1=%.3f, acc1=%.3f'%(i+1, j+1, bat_per_epo, d_loss1,acc1))
    print('>%d, %d/%d, d1=%.3f, acc1=%.3f ,g=%.3f'%(i+1, j+1, bat_per_epo, d_loss1,acc1, g_loss))

In [ ]:
def plot_images(images, n):
    plt.figure(figsize=(20,20))
    for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        plt.imshow(images[i, :, :])
    plt.show()

In [ ]:
latent_points = generate_latent_points(latent_dim, 128)
predicted_images = gen_model.predict(latent_points)
print("Generated Images")
# plot the result
plot_images(predicted_images.clip(0,1), 4)
print("\n\n\n\nReal Images")
plot_images(next(iter(train_ds)), 4)